# pip install prefect transformers peft accelerate bitsandbytes

In [1]:
from prefect import task

# Load Documents
@task(retries=3)
def load_documents(path: str):
    from pathlib import Path

    docs = []
    for file in Path(path).glob("*.txt"):
        docs.append(file.read_text(encoding="utf-8"))

    return docs

In [2]:
# Chunk Documents
@task
def chunk_documents(docs, chunk_size=500, overlap=50):
    chunks = []
    for doc in docs:
        for i in range(0, len(doc), chunk_size - overlap):
            chunks.append(doc[i:i + chunk_size])
    return chunks

In [3]:
# Embed Chunks
@task(tags=["embeddings", "cpu"])
def embed_chunks(chunks):
    from sentence_transformers import SentenceTransformer

    model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)
    embeddings = model.encode(chunks, show_progress_bar=True)
    return embeddings

In [4]:
# Build Vector Store
@task
def build_vector_store(embeddings):
    import faiss
    import numpy as np

    #dim = embeddings.shape[1]
    #index = faiss.IndexFlatL2(dim)
    #index.add(np.array(embeddings))

    embeddings = np.asarray(embeddings, dtype="float32")

    if embeddings.ndim == 1:
        embeddings = embeddings.reshape(1, -1)

    dim = embeddings.shape[1]

    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)

    print("Vectors in index:", index.ntotal)
    return index

In [5]:
# Retrieve Context
@task
def retrieve(query, index, chunks, k=3):
    import numpy as np

    query_embedding = embed_chunks.fn([query])[0]
    D, I = index.search(np.array([query_embedding]), k)
    return [chunks[i] for i in I[0]]

#Generate Answers
@task(tags=["generation", "llm"])
def generate_answer(query, contexts):
    from transformers import pipeline

    generator = pipeline(
        "text-generation",
        model="microsoft/phi-3-mini-4k-instruct",
        device_map="auto"
    )

    prompt = f"""
    Use the context below to answer the question.

    Context:
    {''.join(contexts)}

    Question:
    {query}
    """

    output = generator(prompt, max_new_tokens=200)
    return output[0]["generated_text"]

In [6]:
# Define the Flow
from prefect import flow

@flow(name="rag-pipeline")
def rag_pipeline(doc_path: str, question: str):
    docs = load_documents(doc_path)
    chunks = chunk_documents(docs)
    embeddings = embed_chunks(chunks)
    index = build_vector_store(embeddings)

    contexts = retrieve(question, index, chunks)
    answer = generate_answer(question, contexts)

    print(answer)



In [7]:
# Run the pipeline
rag_pipeline("docs/", "What does this document say about LangChain?")

20:17:50.911 | INFO    | prefect - Starting temporary server on http://127.0.0.1:8356
See https://docs.prefect.io/v3/concepts/server#how-to-guides for more information on running a dedicated Prefect server.

20:17:58.749 | INFO    | Flow run 'sturdy-ibis' - Beginning flow run 'sturdy-ibis' for flow 'rag-pipeline'

20:17:59.105 | INFO    | Task run 'load_documents-858' - Finished in state Completed()

20:17:59.116 | INFO    | Task run 'chunk_documents-35b' - Finished in state Completed()

20:18:15.243 | WARNING | transformers_modules.nomic_hyphen_ai.nomic_hyphen_bert_hyphen_2048.7710840340a098cfb869c4f65e87cf2b1b70caca.modeling_hf_nomic_bert - <All keys matched successfully>

...

20:18:25.190 | WARNING | accelerate.big_modeling - Some parameters are on the meta device because they were offloaded to the disk and cpu.

Device set to use cpu


20:44:31.956 | INFO    | Task run 'generate_answer-5fb' - Finished in state Completed()


    Use the context below to answer the question.

    Context:
    LangChain is an open-source orchestration framework designed to simplify the building of applications using large language models (LLMs). It provides tools and components to connect LLMs with external data sources, APIs, and computational resources, enabling developers to create complex, multi-step AI workflows. 
Key Concepts
The framework is based on the idea of "chaining" different components together to create a cohesive workflow. 
Chains: A sequence of actions or calls (to an LLM, a tool, or for LangChain that involves retrieving relevant information from external documents or databases (like vector stores) to augment the LLM's knowledge and improve the accuracy of its responses.  chatbots.
Agents: An agent uses an LLM as a reasoning engine to autonomously decide which sequence of actions or "tools" to take to accomplish a goal, rather than following a hardcoded sequence.
Tools: Functions or APIs that agents can c

20:44:32.176 | INFO    | Flow run 'sturdy-ibis' - Finished in state Completed()

In [8]:
print("FLOW FINISHED — waiting before exit")
import time
time.sleep(10)

FLOW FINISHED — waiting before exit
